

Na tarefa anterior se fixo unha pequena base de datos con caras. Neste caso se trata de facer unha aplicación de consola que cargue esa base de datos e tente identificar as caras presentes.

Á saída debera escribir os nomes das persoas que foi capaz de identificar e o número de persoas que non foi capaz de poñerlle nome.


Para salvar e cargar as caras:

import pandas as pd
display(",".join(face_descriptorPhill.astype(str)))
csv = pd.DataFrame(data={'nome':['Phill'], 'fingerprint':[",".join(face_descriptorPhill.astype(str))]})
display(csv)

csv.to_csv('proba.csv')

leido = pd.read_csv('proba.csv')
display(leido)
leido["f2"] = leido.apply(lambda row: np.array(row['fingerprint'].split(',')).astype(np.float64), axis=1)




### Importación de librerías y imagen grupal

In [5]:
import cv2
import dlib
import numpy as np
import matplotlib as plt

imagen_grupal = cv2.imread("sam-frodo-gollum.png")
imaxeTodosColor = cv2.cvtColor(imagen_grupal, cv2.COLOR_BGR2RGB)

### Importación de herramientas de reconocimiento facial 

In [6]:

predictor_path ="./shape_predictor_5_face_landmarks.dat"
face_rec_model_path = "./dlib_face_recognition_resnet_model_v1.dat"


detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

known_face_encodings = []
known_face_names = []

In [7]:
face_names = []

dets = detector(imaxeTodosColor, 1)
tolerancia = 0.6
for d in dets:

    shape = sp(imaxeTodosColor,d)
    face_descriptor = np.array(facerec.compute_face_descriptor(imaxeTodosColor, shape))
    print(np.linalg.norm(face_descriptor - known_face_encodings[0][0]))
    # See if the face is a match for the known face(s)
    matches = [(np.linalg.norm(face_descriptor - match[0]), match[1]) for match in known_face_encodings if np.linalg.norm(face_descriptor - match[0]) <= tolerancia]
    print(matches)
    if len(matches) == 0:
        face_names.append(((d.top(), d.right(), d.bottom(), d.left()),"NonSei"))
    else:
        carasOrdenadas = sorted(matches,key=lambda x: x[0]) 
        face_names.append(((d.top(), d.right(), d.bottom(), d.left()),carasOrdenadas[0][1]))
imaxeConMarcas = imaxeTodosColor.copy() 
for (top, right, bottom, left), name in face_names:
    # Scale back up face locations since the frame we detected in was scaled to 1/4 size

    color = (0, 255, 0)
    if name=="NonSei":
        color = (0, 0, 255)
    # Draw a box around the face
    cv2.rectangle(imaxeConMarcas, (left, top), (right, bottom), color, 2)

    # Draw a label with a name below the face
    cv2.rectangle(imaxeConMarcas, (left, bottom + 20), (right, bottom), color, cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(imaxeConMarcas, name, (left + 6, bottom + 15), font, 0.5, (255, 255, 255), 1)

plt.imshow(imaxeConMarcas)
plt.title('Caras atopadas')
plt.show() 

IndexError: list index out of range